# Usage and more of the Calibrations Class [v1.1]

In [7]:
# imports
import os
import glob
import numpy

from matplotlib import pyplot

from pypeit.pypeit import PypeIt

from pypeit import calibrations
from pypeit import pypeitsetup

from pypeit.spectrographs import keck_deimos
from pypeit.spectrographs.slitmask import SlitRegister

In [2]:
os.environ['PYPEIT_DEV']

'/Users/westfall/Work/packages/PypeIt-development-suite'

## Setup

In [6]:
#pypeit_file = os.path.join(os.environ['PYPEIT_DEV'], 'pypeit_files',
#                           'keck_deimos_830g_m.pypeit')
pypeit_file = os.path.join(os.environ['PYPEIT_DEV'], 'REDUX_OUT/Keck_DEIMOS/830G_M_8500/',
                           'keck_deimos_830g_m_8500.pypeit')

In [8]:
rdx = PypeIt(pypeit_file)

[INFO]    :: util.py 483 parse_pypeit_file() - Loading the reduction file
[INFO]    :: util.py 509 parse_pypeit_file() - Found 4 raw data frames
[INFO]    :: util.py 530 parse_pypeit_file() - Input file loaded successfully
[WARNING] :: metadata.py 1219 get_frame_types() - Removing existing frametype and framebit columns.
[INFO]    :: metadata.py 1230 get_frame_types() - Typing files
[INFO]    :: metadata.py 1237 get_frame_types() - Using user-provided frame types.
[INFO]    :: pypeit.py 125 __init__() - Setting reduction path to /Users/westfall/Work/packages/pypeit/doc/nb
[INFO]    :: paths.py 38 make_dirs() - Creating Science directory
[INFO]    :: paths.py 41 make_dirs() - The following directory already exists:
             /Users/westfall/Work/packages/pypeit/doc/nb/Science
[INFO]    :: paths.py 65 make_dirs() - Creating Object directories
[INFO]    :: paths.py 68 make_dirs() - Creating Master Calibrations directory
[INFO]    :: paths.py 72 make_dirs() - The following directory alr

In [ ]:
setupc = pypeitsetup.PypeItSetup.from_pypeit_file(pypeit_file)

In [ ]:
par, spec, fitstbl, setup_dict = setupc.run()

In [ ]:
fsort.make_dirs(spec.spectrograph, par['calibrations']['caldir'], par['rdx']['scidir'],
                par['rdx']['qadir'], overwrite=True)


## Choose the detector, sci_ID

In [ ]:
det = 1
sci_ID = 1
setup = pypsetup.instr_setup(sci_ID, det, fitstbl, setup_dict,
                             spec.detector[det-1]['numamplifiers'],
                             must_exist=True)
setup

##  Calibrations

## Instantiate

In [ ]:
caliBrate = calibrations.MultiSlitCalibrations(fitstbl, spectrograph=spec,
                                               par=par['calibrations'],
                                               save_masters=False, write_qa=False)

In [ ]:
# Setup for a specific science frame + detector
caliBrate.reset(setup, det, sci_ID, par['calibrations'])

## Do it

In [ ]:
_ = caliBrate.get_datasec_img()

In [ ]:
msbias = caliBrate.get_bias()

In [ ]:
msarc = caliBrate.get_arc()

In [ ]:
msbpm = caliBrate.get_bpm()

In [ ]:
pixlocn = caliBrate.get_pixlocn()

In [ ]:
tslits_dict, maskslits = caliBrate.get_slits(arms=True)

In [ ]:
caliBrate.traceSlits.show('edges')

In [ ]:
trace_files = fsort.list_of_files(fitstbl, 'trace', sci_ID)
spec.get_slitmask(trace_files[0])
spec.get_detector_map()

In [ ]:
spec_i = 3900
trace_slit_cen = (caliBrate.traceSlits.lcen[spec_i,:] + caliBrate.traceSlits.rcen[spec_i,:])/2
offset = spec.detector_map.npix[0]/2-spec.detector_map.ccd_center[0,0]
pix_per_mm = spec.telescope.platescale()/spec.detector[0]['platescale']
register = SlitRegister(trace_slit_cen, spec.slitmask.center[:,0],
                        guess_offset=offset, guess_scale=pix_per_mm,
                        offset_limits=[offset-100,offset+100],
                        scale_limits=[pix_per_mm/1.1, pix_per_mm*1.1],
                        fit=True)

In [ ]:
missing = register.missing_from_trace()
mask_pix = register.mask_to_trace_coo()
pyplot.scatter(trace_slit_cen, mask_pix[register.match_index], color='k', zorder=1,
               label='Found')
if len(missing) > 0:
    print('Found {0} missing slits: {1}'.format(len(missing), missing))
    pyplot.scatter(mask_pix[missing], mask_pix[missing], color='r', zorder=1,
                   label='Missing')
pyplot.xlabel('Trace X (pixel)')
pyplot.ylabel('Best-Fit Mask X (pixel)')
pyplot.legend()
pyplot.show()

In [ ]:
lr_offset = numpy.array([-0.5,0.5])[None,:]*spec.slitmask.length[missing][:,None]*register.par[1]
new_slits = numpy.append(mask_pix[missing][:,None] + lr_offset,
                         numpy.array([spec_i]*len(missing)).reshape(-1,1), axis=1).astype(int)
new_slits = new_slits[numpy.argsort(new_slits[:,0]),:]
pyplot.scatter(caliBrate.traceSlits.lcen[spec_i,:], caliBrate.traceSlits.rcen[spec_i,:],
               color='k', marker='.', s=30, lw=0)
pyplot.scatter(new_slits[:,0], new_slits[:,1],
               color='r', marker='.', s=30, lw=0)
new_slits

In [ ]:
#caliBrate.traceSlits.add_user_slits([new_slits[0,:]], run_to_finish=True)
caliBrate.traceSlits.add_user_slits(new_slits, run_to_finish=True)

In [ ]:
caliBrate.traceSlits.show('edges')
print('fubar')